In [1]:
!pip install networkx matplotlib

In [5]:
import re
import numpy as np
import networkx as nx
import json
from scipy.sparse import lil_matrix, save_npz

# ===== Step 1: 解析 Verilog 檔案 =====
def parse_verilog(verilog_code):
    gates = []

    for line in verilog_code.splitlines():
        line = line.strip()
        print(f"Processing line: {line}")

        # 解析 BUF gate
        buf_match = re.match(r'^\s*buf\s+(\S+)\((\S+),\s*(\S+)\);', line)
        if buf_match:
            gates.append(('BUF', buf_match.group(1), buf_match.group(2), buf_match.group(3)))  # buf, output, input
            continue

        # 解析 DFF gate（具名端口的 DFF gate，處理 .RN, .SN 等）
        dff_match = re.match(r'^\s*dff\s+(\S+)\s*\(\.RN\(([^)]+)\),\s*\.SN\(([^)]+)\),\s*\.CK\(([^)]+)\),\s*\.D\(([^)]+)\),\s*\.Q\(([^)]+)\)\);', line)
        if dff_match:
            gates.append(('DFF', dff_match.group(1), dff_match.group(2), dff_match.group(3),
                          dff_match.group(4), dff_match.group(5), dff_match.group(6)))  # dff, RN, SN, CK, D, Q
            continue

        # 解析 OR gate (雙輸入 gate)
        or_match = re.match(r'^\s*or\s+(\S+)\((\S+)\s*,\s*(\S+)\s*,\s*(\S+)\);', line)
        if or_match:
            gates.append(('OR', or_match.group(1), or_match.group(2), or_match.group(3), or_match.group(4)))  # or, output, input1, input2
            continue

        # 解析 NOR gate (雙輸入 gate)
        nor_match = re.match(r'^\s*nor\s+(\S+)\((\S+)\s*,\s*(\S+)\s*,\s*(\S+)\);', line)
        if nor_match:
            gates.append(('NOR', nor_match.group(1), nor_match.group(2), nor_match.group(3), nor_match.group(4)))  # nor, output, input1, input2
            continue

        # 解析 NOT gate (單輸入 gate)
        not_match = re.match(r'^\s*not\s+(\S+)\((\S+)\s*,\s*(\S+)\);', line)
        if not_match:
            gates.append(('NOT', not_match.group(1), not_match.group(2), not_match.group(3)))  # not, output, input
            continue

        # 解析 XOR gate (雙輸入 gate)
        xor_match = re.match(r'^\s*xor\s+(\S+)\((\S+)\s*,\s*(\S+)\s*,\s*(\S+)\);', line)
        if xor_match:
            gates.append(('XOR', xor_match.group(1), xor_match.group(2), xor_match.group(3), xor_match.group(4)))  # xor, output, input1, input2
            continue

        # 解析 AND gate (雙輸入 gate)
        and_match = re.match(r'^\s*and\s+(\S+)\((\S+)\s*,\s*(\S+)\s*,\s*(\S+)\);', line)
        if and_match:
            gates.append(('AND', and_match.group(1), and_match.group(2), and_match.group(3), and_match.group(4)))  # and, output, input1, input2
            continue

        # 解析 NAND gate (雙輸入 gate)
        nand_match = re.match(r'^\s*nand\s+(\S+)\((\S+)\s*,\s*(\S+)\s*,\s*(\S+)\);', line)
        if nand_match:
            gates.append(('NAND', nand_match.group(1), nand_match.group(2), nand_match.group(3), nand_match.group(4)))  # nand, output, input1, input2
            continue

    #print(gates)
    return gates


# ===== Step 2: 轉換成 infolist 格式 =====
def gates_to_infolist(gates, trojan_gates=[]):
    infolist = []
    for g in gates:
        gtype = g[0]
        instname = g[1]
        output = g[2]
        inputs = list(g[3:])

        portnames = ['Y'] + [f'A{i+1}' for i in range(len(inputs))]
        connnames = [output] + inputs

        is_trojan = (instname in trojan_gates or output in trojan_gates or any(inp in trojan_gates for inp in inputs))
        infolist.append((
            gtype, gtype, instname, instname, portnames, connnames, is_trojan
        ))
    return infolist

# ===== Step 3: 建立 adjacency matrix & features =====
def build_lookup(infolist):
    lookup = {}
    for i, info in enumerate(infolist):
        conns = info[5]  # connection names
        for conn in conns[1:]:  # skip output
            if conn not in lookup:
                lookup[conn] = []
            lookup[conn].append(i)
    return lookup

def build_graph_features(infolist):
    numnodes = len(infolist)
    adj = lil_matrix((numnodes, numnodes), dtype=bool)
    class_map = {}
    train_indices = list(range(numnodes))  # 全部都當 train

    gatelist = sorted(list(set([x[0] for x in infolist])))
    gatelookup = {g: i for i, g in enumerate(gatelist)}

    feats = np.zeros((numnodes, len(gatelist) + 2))
    lookup = build_lookup(infolist)

    gate_map = {}  # <== 新增這行

    for i, info in enumerate(infolist):
        gatetype = info[0]
        conns = info[5]
        feats[i][gatelookup[gatetype]] = 1

        for conn in conns[1:]:  # skip output
            if conn in lookup:
                for j in lookup[conn]:
                    if i != j:
                        adj[i, j] = True
                        feats[i][-1] += 1  # out
                        feats[j][-2] += 1  # in

        class_map[i] = 1 if info[6] else 0
        gate_map[i] = info[2]  # <== index i 對應 gate 名稱

    return adj, feats, train_indices, class_map, gate_map


# ===== Step 4: 儲存 GraphSAGE 所需格式 =====
def save_graphsage_format(adj, feats, class_map, train_indices, gate_map):
    save_npz("adj_full.npz", adj.tocsr())
    save_npz("adj_train.npz", adj.tocsr())

    np.save("feats.npy", feats, allow_pickle=False)

    with open("class_map.json", "w") as f:
        json.dump(class_map, f)

    with open("role.json", "w") as f:
        json.dump({'tr': train_indices, 'va': [], 'te': []}, f)

    with open("gate_map.json", "w") as f:  # <== 新增這段
        json.dump(gate_map, f)


# ===== 主流程 =====
def process_single_verilog(filepath, gt_trojan_filepath):
    with open(filepath, 'r') as f:
        code = f.read()

    gates = parse_verilog(code)
    # 若無trojan gates, txt只有一行: NO_TROJAN
    # 若有trojan gates, txt第一行是 "TROJANED", 第二行是 "TROJAN_GATES", 最後一行是 "END_TROJAN_GATES"
    trojan_gates = []
    with open(gt_trojan_filepath, 'r') as f:
        lines = [l.strip() for l in f]
        if lines and lines[0] == "TROJANED":
            for line in lines[2:]:
                if line == "END_TROJAN_GATES":
                    break
                trojan_gates.append(line)
        else:
            trojan_gates = []
    infolist = gates_to_infolist(gates, trojan_gates)
    adj, feats, train_indices, class_map, gate_map = build_graph_features(infolist)
    save_graphsage_format(adj, feats, class_map, train_indices, gate_map)


    print("✅ Graph feature files saved.")

if __name__ == "__main__":
    process_single_verilog("design4.v", "result4.txt")  # 改成你的檔案



Processing line: module top(n0, n1, n2, n3, n4, n5);
Processing line: input n0, n1;
Processing line: input [127:0] n2;
Processing line: output [63:0] n3, n4, n5;
Processing line: wire n0, n1;
Processing line: wire [127:0] n2;
Processing line: wire [63:0] n3, n4, n5;
Processing line: wire [63:0] n6;
Processing line: wire [63:0] n7;
Processing line: wire [63:0] n8;
Processing line: wire [63:0] n9;
Processing line: wire [63:0] n10;
Processing line: wire [63:0] n11;
Processing line: wire [2:0] n12;
Processing line: wire [3:0] n13;
Processing line: wire [15:0] n14;
Processing line: wire n15, n16, n17, n18, n19, n20, n21, n22;
Processing line: wire n23, n24, n25, n26, n27, n28, n29, n30;
Processing line: wire n31, n32, n33, n34, n35, n36, n37, n38;
Processing line: wire n39, n40, n41, n42, n43, n44, n45, n46;
Processing line: wire n47, n48, n49, n50, n51, n52, n53, n54;
Processing line: wire n55, n56, n57, n58, n59, n60, n61, n62;
Processing line: wire n63, n64, n65, n66, n67, n68, n69, n70;